Why are we taking time to learn a process that is already automated? The answer is that learning to do the calculations by hand will give us insight into how tf-idf really works. This insight is valuable. Instead of viewing tf-idf scores as some magical number our spreadsheet or computer program gives us, we'll be able to explain where that number comes from.

In [646]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math
import operator
import statistics
from string import punctuation
stop_words = set(stopwords.words('spanish') + list(punctuation))
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [647]:

def get_text_from_file(fname):
    """
    Get text from a text document.
    """
    try:
        with open(fname, 'r', encoding='utf-8') as f:
            text = f.read()
        return text
    except Exception as e:
        print(f"Error reading the file: {e}")
        return None


In [648]:
def remove_string_special_characters(s):
    """
    This function removes special characters from within a string.
    parameters: 
        s(str): single input string.
    return: 
        stripped(str): A string with special characters removed.
    """

    # Replace special character with ' '
    stripped = re.sub('[^\w\s]', '', s)
    stripped = re.sub('_', '', stripped)

    # Change any whitespace to one space
    stripped = re.sub('\s+', ' ', stripped)

    # Remove start and end white spaces
    stripped = stripped.strip()
    
    return stripped

In [649]:
def count_words(text):
    """This function returns the 
    total number of words in the input text.
    """
    count = 0
    words = word_tokenize(text)
    for word in words:
        count += 1
    return count

In [650]:
def get_doc(text_sents_clean):
    """
    this function splits the text into sentences and
    considering each sentence as a document, calculates the 
    total word count of each.
    """
    doc_info = []
    i = 0
    for sent in text_sents_clean:
        i += 1 
        count = count_words(sent)
        temp = {'doc_id' : i, 'doc_length' : count}
        doc_info.append(temp)
    return doc_info

In [651]:
def create_freq_dict(sents):
    """
    This function creates a frequency dictionary
    of each document that contains words other than
    stop words.
    """
    i = 0
    freqDict_list = []
    for sent in sents:
        i += 1
        freq_dict = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word not in stop_words:
                if word in freq_dict:
                    freq_dict[word] += 1
                else:
                    freq_dict[word] = 1
                temp = {'doc_id' : i, 'freq_dict': freq_dict}
        freqDict_list.append(temp)
    return freqDict_list

In [652]:
def global_frequency(text_sents_clean):
    """
    This function returns a dictionary with the frequency 
    count of every word in the text
    """
    freq_table = {}
    text = ' '.join(text_sents_clean) #join the cleaned sentences to get the text 
    words = word_tokenize(text)
    for word in words:
        word = word.lower()
        word = ps.stem(word)
        if word not in stop_words:
            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1
                
    return freq_table

In [653]:
def get_keywords(text_sents_clean):
    """
    This function gets the top 5 most
    frequently occuring words in the whole text
    and stores them as keywords
    """
    freq_table = global_frequency(text_sents_clean)
    #sort in descending order
    freq_table_sorted = sorted(freq_table.items(), key = operator.itemgetter(1), reverse = True) 
    print(freq_table_sorted)
    keywords = []
    for i in range(0, 10):  #taking first 5 most frequent words
        keywords.append(freq_table_sorted[i][0])
    return keywords

In [654]:
def computeTF(doc_info, freqDict_list):
    """
    tf = (frequency of the term in the doc/total number of terms in the doc)
    """
    TF_scores = []
    
    for tempDict in freqDict_list:
        id = tempDict['doc_id']
        for k in tempDict['freq_dict']:
            temp = {'doc_id' : id,
                    'TF_score' : tempDict['freq_dict'][k]/doc_info[id-1]['doc_length'],
                   'key' : k}
            TF_scores.append(temp)
    return TF_scores

In [655]:
def computeIDF(doc_info, freqDict_list):
    """
    idf = ln(total number of docs/number of docs with term in it)
    """
    
    IDF_scores = []
    counter = 0
    for dict in freqDict_list:
        counter += 1
        for k in dict['freq_dict'].keys():
            count = sum([k in tempDict['freq_dict'] for tempDict in freqDict_list])
            temp = {'doc_id' : counter, 'IDF_score' : math.log(len(doc_info)/count), 'key' : k}
    
            IDF_scores.append(temp)
                
    return IDF_scores

Before computing the word frequency, we must clean the data- remove punctuation 
and special characters.


In [671]:

text = get_text_from_file('ia.txt')
text_sents = sent_tokenize(text)
text_sents_clean = [remove_string_special_characters(s) for s in text_sents] #if s.istitle() == False]
doc_info = get_doc(text_sents_clean)
print(doc_info)

[{'doc_id': 1, 'doc_length': 27}, {'doc_id': 2, 'doc_length': 15}, {'doc_id': 3, 'doc_length': 20}, {'doc_id': 4, 'doc_length': 13}, {'doc_id': 5, 'doc_length': 0}, {'doc_id': 6, 'doc_length': 20}, {'doc_id': 7, 'doc_length': 30}, {'doc_id': 8, 'doc_length': 26}, {'doc_id': 9, 'doc_length': 23}]


In [672]:
freqDict_list = create_freq_dict(text_sents_clean)
print(freqDict_list)
TF_scores = computeTF(doc_info, freqDict_list)
print(TF_scores)
IDF_scores = computeIDF(doc_info, freqDict_list)

[{'doc_id': 1, 'freq_dict': {'actual': 1, 'sociedad': 1, 'cambiant': 1, 'sector': 1, 'distribución': 1, 'enfrenta': 1, 'numeroso': 1, 'desafío': 1, 'presion': 1, 'competitiva': 1, 'asociada': 1, 'lucha': 1, 'manten': 1, 'márgene': 1, 'ajustado': 1}}, {'doc_id': 2, 'freq_dict': {'tormenta': 1, 'priorizar': 1, 'dond': 1, 'toma': 1, 'gran': 1, 'protagonismo': 1, 'capacidad': 1, 'reacción': 1, 'retail': 1}}, {'doc_id': 3, 'freq_dict': {'pued': 1, 'resultar': 1, 'difícil': 1, 'identificar': 1, 'aquella': 1, 'tecnología': 1, 'proceso': 1, 'contribuirán': 1, 'creación': 1, 'ventaja': 1, 'competitiva': 1, 'distribución': 1}}, {'doc_id': 4, 'freq_dict': {'obstant': 1, 'entr': 1, 'toda': 1, 'tecnología': 1, 'pueden': 1, 'ayudar': 1, 'resolv': 1, 'problema': 1}}, {'doc_id': 4, 'freq_dict': {'obstant': 1, 'entr': 1, 'toda': 1, 'tecnología': 1, 'pueden': 1, 'ayudar': 1, 'resolv': 1, 'problema': 1}}, {'doc_id': 6, 'freq_dict': {'inteligencia': 1, 'artifici': 1, 'obtien': 1, 'mayor': 1, 'impacto': 1,

In [658]:
def computeTFIDF(TF_scores, IDF_scores):
    """
    TFIDF is computed by multiplying the coressponding
    TF and IDF values of each term. 
    """
    TFIDF_scores = []
    for j in IDF_scores:
        for i in TF_scores:
            if j['key'] == i['key'] and j['doc_id'] == i['doc_id']:
                temp = {'doc_id' : i['doc_id'],
                        'TFIDF_score' : j['IDF_score']*i['TF_score'],
                       'key' : i['key']}
        TFIDF_scores.append(temp)
    return TFIDF_scores

In [659]:
TFIDF_scores = computeTFIDF(TF_scores, IDF_scores)

In [660]:
def weigh_keywords(TFIDF_scores):
    """
    This function doubles the TFIDF score
    of the words that are keywords
    """
    keywords = get_keywords(text_sents_clean)
    for temp_dict in TFIDF_scores:
        if temp_dict['key'] in keywords:
            temp_dict['TFIDF_score'] *= 2
    return TFIDF_scores

In [661]:
TFIDF_scores = weigh_keywords(TFIDF_scores)

[('ia', 39), ('inteligencia', 13), ('símbolo', 13), ('gener', 12), ('inteligent', 10), ('sistema', 10), ('fuert', 9), ('modelo', 9), ('hipótesi', 9), ('débil', 8), ('ordenador', 8), ('ajedrez', 8), ('est', 7), ('específica', 7), ('ejemplo', 7), ('físico', 7), ('jugar', 7), ('objetivo', 6), ('tipo', 6), ('humano', 6), ('mediant', 6), ('ser', 6), ('caso', 6), ('sobr', 5), ('máquina', 5), ('hecho', 5), ('humana', 5), ('sere', 5), ('ssf', 5), ('decir', 5), ('programa', 5), ('relacionado', 4), ('último', 4), ('má', 4), ('origen', 4), ('newel', 4), ('simon', 4), ('dama', 4), ('capacidad', 4), ('searl', 4), ('entorno', 4), ('real', 4), ('artifici', 3), ('entr', 3), ('aspecto', 3), ('conocimiento', 3), ('ambicioso', 3), ('construir', 3), ('grand', 3), ('dificultad', 3), ('similar', 3), ('científico', 3), ('conoc', 3), ('estructura', 3), ('siglo', 3), ('si', 3), ('actualment', 3), ('artículo', 3), ('motivo', 3), ('ture', 3), ('1975', 3), ('cabo', 3), ('acuerdo', 3), ('pueden', 3), ('proceso', 3

In [662]:
def get_sent_score(TFIDF_scores, text_sents, doc_info):
    """
    This function prints out the summary and returns the 
    score of each sentence in a list.
    
    The score of a sentence is calculated by adding the TFIDF
    scores of the words that make up the sentence.
    """
    sentence_info = []
    for doc in doc_info:
        """
        This loops through each document(sentence)
        and calculates their 'sent_score'
        """
        sent_score = 0
        for i in range(0, len(TFIDF_scores)):
            temp_dict = TFIDF_scores[i]
            if doc['doc_id'] == temp_dict['doc_id']:
                sent_score += temp_dict['TFIDF_score']
        temp = {'doc_id' : doc['doc_id'], 'sent_score' : sent_score,
                'sentence' : text_sents[doc['doc_id']-1]}
        sentence_info.append(temp)
    print(sentence_info)
    return sentence_info

In [663]:
sentence_info = get_sent_score(TFIDF_scores, text_sents, doc_info)

[{'doc_id': 1, 'sent_score': 2.730919029803735, 'sentence': 'El futuro de la IA: hacia inteligencias artificiales realmente inteligentes\nRamón López de Mántaras\nInstituto de Investigación en Inteligencia Artificial (IIIA), Bellaterra, España\nEste capítulo contiene algunas reflexiones sobre inteligencia artificial (IA).'}, {'doc_id': 2, 'sent_score': 1.828606444750739, 'sentence': 'En primer lugar, se explica la distinción entre la IA fuerte y la débil, así como los conceptos relacionados de IA general y específica, dejando claro que todas las manifestaciones existentes de IA son débiles y específicas.'}, {'doc_id': 3, 'sent_score': 1.899697459059781, 'sentence': 'Se describen brevemente los principales modelos, insistiendo en la importancia de la corporalidad como aspecto clave para conseguir una IA de naturaleza general.'}, {'doc_id': 4, 'sent_score': 1.8221646140827097, 'sentence': 'A continuación se aborda la necesidad de proporcionar a las máquinas conocimientos de sentido común

In [664]:
def get_summary(sentence_info):
    sum = 0
    summary = []
    array = []
    for temp_dict in sentence_info:
        """
        This loop gets the sum of scores
        of all the sentences.
        """
        sum += temp_dict['sent_score']
    avg = sum/len(sentence_info) #computing the average tf-idf score
    for temp_dict in sentence_info:
        """
        This loop gets the sentence scores 
        and stores them in an array.
        """
        array.append(temp_dict['sent_score'])
    stdev = statistics.stdev(array) #computing standard deviation on the array   
    for sent in sentence_info:
        """
        This loop is for getting the sumamry by 
        extracting sentences by an if clause
        """
        if(sent['sent_score']) >= avg: # + 1.5*stdev:
            summary.append(sent['sentence'])
    summary = '\n'.join(summary)
    return summary

In [665]:
summary = get_summary(sentence_info)
print(summary)
print(len(text))
print(len(summary))

El futuro de la IA: hacia inteligencias artificiales realmente inteligentes
Ramón López de Mántaras
Instituto de Investigación en Inteligencia Artificial (IIIA), Bellaterra, España
Este capítulo contiene algunas reflexiones sobre inteligencia artificial (IA).
En primer lugar, se explica la distinción entre la IA fuerte y la débil, así como los conceptos relacionados de IA general y específica, dejando claro que todas las manifestaciones existentes de IA son débiles y específicas.
Se describen brevemente los principales modelos, insistiendo en la importancia de la corporalidad como aspecto clave para conseguir una IA de naturaleza general.
A continuación se aborda la necesidad de proporcionar a las máquinas conocimientos de sentido común que hagan posible avanzar hacia el ambicioso objetivo de construir IA de tipo general.
A lo largo de los últimos siglos, este afán por construir máquinas inteligentes nos ha conducido a inventar modelos o metáforas del cerebro humano.
Por ejemplo, en el

In [666]:
text2 = summary
text_sents2 = sent_tokenize(text2)
text_sents_clean2 = [remove_string_special_characters(s) for s in text_sents2] #if s.istitle() == False]
doc_info2 = get_doc(text_sents_clean2)
freqDict_list2 = create_freq_dict(text_sents_clean2)
TF_scores2 = computeTF(doc_info2, freqDict_list2)
IDF_scores2 = computeIDF(doc_info2, freqDict_list2)
TFIDF_scores2 = computeTFIDF(TF_scores2, IDF_scores2)
TFIDF_scores2 = weigh_keywords(TFIDF_scores2)
sentence_info2 = get_sent_score(TFIDF_scores2, text_sents2, doc_info2)
summary2 = get_summary(sentence_info2)

[('ia', 39), ('inteligencia', 13), ('símbolo', 13), ('gener', 12), ('inteligent', 10), ('sistema', 10), ('fuert', 9), ('modelo', 9), ('hipótesi', 9), ('débil', 8), ('ordenador', 8), ('ajedrez', 8), ('est', 7), ('específica', 7), ('ejemplo', 7), ('físico', 7), ('jugar', 7), ('objetivo', 6), ('tipo', 6), ('humano', 6), ('mediant', 6), ('ser', 6), ('caso', 6), ('sobr', 5), ('máquina', 5), ('hecho', 5), ('humana', 5), ('sere', 5), ('ssf', 5), ('decir', 5), ('programa', 5), ('relacionado', 4), ('último', 4), ('má', 4), ('origen', 4), ('newel', 4), ('simon', 4), ('dama', 4), ('capacidad', 4), ('searl', 4), ('entorno', 4), ('real', 4), ('artifici', 3), ('entr', 3), ('aspecto', 3), ('conocimiento', 3), ('ambicioso', 3), ('construir', 3), ('grand', 3), ('dificultad', 3), ('similar', 3), ('científico', 3), ('conoc', 3), ('estructura', 3), ('siglo', 3), ('si', 3), ('actualment', 3), ('artículo', 3), ('motivo', 3), ('ture', 3), ('1975', 3), ('cabo', 3), ('acuerdo', 3), ('pueden', 3), ('proceso', 3

In [667]:
print(summary2)
print(len(summary2))

El futuro de la IA: hacia inteligencias artificiales realmente inteligentes
Ramón López de Mántaras
Instituto de Investigación en Inteligencia Artificial (IIIA), Bellaterra, España
Este capítulo contiene algunas reflexiones sobre inteligencia artificial (IA).
Por ejemplo, en el siglo XVII, Descartes se preguntó si un complejo sistema mecánico compuesto de engranajes, poleas y tubos podría, en principio, emular el pensamiento.
Un SSF consiste en un conjunto de entidades denominadas símbolos que, mediante relaciones, pueden ser combinados formando estructuras más grandes —como los átomos que se combinan formando moléculas— y que pueden ser transformados aplicando un conjunto de procesos.
Estos procesos pueden generar nuevos símbolos, crear y modificar relaciones entre símbolos, almacenar símbolos, comparar si dos símbolos son iguales o distintos, etcétera.
En definitiva, de acuerdo con la hipótesis SSF, la naturaleza del sustrato (circuitos electrónicos o redes neuronales) carece de impo

In [668]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

def summarize_text(text, num_sentences=10):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum() and word.lower() not in stopwords.words('spanish')]

    freq_dist = FreqDist(words)
    key_sentences = sorted(sentences, key=lambda sentence: sum(freq_dist[word] for word in word_tokenize(sentence.lower()) if word.isalnum()), reverse=True)[:num_sentences]

    return ' '.join(key_sentences)

# Ejemplo de uso:
texto = get_text_from_file('texto.txt')
resumen = summarize_text(texto)
print(resumen)
print(len(resumen))

LA HIPÓTESIS DEL SISTEMA DE SÍMBOLOS FÍSICOS: IA DÉBIL VERSUS IA FUERTE
En una ponencia, con motivo de la recepción del prestigioso Premio Turing en 1975, Allen Newell y Herbert Simon (Newell y Simon, 1975) formularon la hipótesis del Sistema de Símbolos Físicos según la cual «todo sistema de símbolos físicos posee los medios necesarios y suficientes para llevar a cabo acciones inteligentes». En primer lugar, se explica la distinción entre la IA fuerte y la débil, así como los conceptos relacionados de IA general y específica, dejando claro que todas las manifestaciones existentes de IA son débiles y específicas. El diseño y realización de inteligencias artificiales que únicamente muestran comportamiento inteligente en un ámbito muy específico, está relacionado con lo que se conoce por IA débil en contraposición con la IA fuerte a la que, de hecho, se referían Newell y Simon y otros padres fundadores de la IA. Ello no significa que la IA simbólica no pueda ser usada para, por ejemplo, 